# Assignment 4: Named entity recognition. Мария Терехина, 152 группа

Построить модель для обнаружения и классификации именованных сущностей (named entities). На базе корпуса CoNLL 2002.  

Используйте в своем решении ансамбли над решающими деревьями: RandomForest, Gradient Boosting (xgboost, lightgbm, catboost) 
Tutorials:  
1. https://github.com/Microsoft/LightGBM/tree/master/examples/python-guide
1. https://github.com/catboost/tutorials 


Чем больше baseline'ов вы превзойдете, тем выше ваша оценка
Метрика качества f1 (f1_macro) (чем выше, тем лучше)
 
baseline 1: 0.0604      random labels  
baseline 2: 0.3966      PoS features + logistic regression  
baseline 3: 0.8122      word2vec cbow embedding + baseline 2 + svm    

! Your results must be reproducible. Если ваша модель - стохастическая, то вы явно должны задавать все seed и random_state в параметрах моделей   

bonus, think about:  
1. How can you exploit that words belong to some sentence?
2. Why we selected f1 score with macro averaging as our classification quality measure? What other metrics are suitable?   

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


SEED=1337

In [2]:
df = pd.read_csv('ner_short.csv', index_col=0)
df.head()

next-next-pos next-next-word next-pos      next-word  pos    prev-pos  \
0           NNS  demonstrators       IN             of  NNS  __START1__   
1           VBP           have      NNS  demonstrators   IN         NNS   
2           VBN        marched      VBP           have  NNS          IN   
3            IN        through      VBN        marched  VBP         NNS   
4           NNP         London       IN        through  VBN         VBP   

  prev-prev-pos prev-prev-word      prev-word  sentence_idx           word tag  
0    __START2__     __START2__     __START1__           1.0      Thousands   O  
1    __START1__     __START1__      Thousands           1.0             of   O  
2           NNS      Thousands             of           1.0  demonstrators   O  
3            IN             of  demonstrators           1.0           have   O  
4           NNS  demonstrators           have           1.0        marched   O

In [3]:
# number of sentences
df.sentence_idx.max()

1500.0

In [4]:
# class distribution
df.tag.value_counts(normalize=True )

O        0.852828
B-geo    0.027604
B-gpe    0.020935
B-org    0.020247
I-per    0.017795
B-tim    0.016927
B-per    0.015312
I-org    0.013937
I-geo    0.005383
I-tim    0.004247
B-art    0.001376
I-gpe    0.000837
I-art    0.000748
B-eve    0.000628
I-eve    0.000508
B-nat    0.000449
I-nat    0.000239
Name: tag, dtype: float64

In [5]:
# sentence length
tdf = df.set_index('sentence_idx')
tdf['length'] = df.groupby('sentence_idx').tag.count()
df = tdf.reset_index(drop=False)

In [6]:
# encode categorial variables

le = LabelEncoder()
df['pos'] = le.fit_transform(df.pos)
df['next-pos'] = le.fit_transform(df['next-pos'])
df['next-next-pos'] = le.fit_transform(df['next-next-pos'])
df['prev-pos'] = le.fit_transform(df['prev-pos'])
df['prev-prev-pos'] = le.fit_transform(df['prev-prev-pos'])

In [7]:
df.head()

sentence_idx  next-next-pos next-next-word  next-pos      next-word  pos  \
0           1.0             18  demonstrators         9             of   18   
1           1.0             33           have        18  demonstrators    9   
2           1.0             32        marched        33           have   18   
3           1.0              9        through        32        marched   33   
4           1.0             16         London         9        through   32   

   prev-pos  prev-prev-pos prev-prev-word      prev-word           word tag  \
0        39             40     __START2__     __START1__      Thousands   O   
1        18             39     __START1__      Thousands             of   O   
2         9             18      Thousands             of  demonstrators   O   
3        18              9             of  demonstrators           have   O   
4        33             18  demonstrators           have        marched   O   

   length  
0      48  
1      48  
2      48  
3      48  
4      48

In [9]:
# splitting
y = LabelEncoder().fit_transform(df.tag)

df_train, df_test, y_train, y_test = model_selection.train_test_split(df, y, stratify=y, 
                                                                      test_size=0.25, random_state=SEED, shuffle=True)
print('train', df_train.shape[0])
print('test', df_test.shape[0])

train 50155
test 16719


In [22]:
# some wrappers to work with word2vec
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from collections import defaultdict

   
class Word2VecWrapper(TransformerMixin):
    def __init__(self, window=5,negative=5, size=100, iter=100, is_cbow=False, random_state=SEED):
        self.window_ = window
        self.negative_ = negative
        self.size_ = size
        self.iter_ = iter
        self.is_cbow_ = is_cbow
        self.w2v = None
        self.random_state = random_state
        
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        self.w2v = Word2Vec(sentences_list, 
                            window=self.window_,
                            negative=self.negative_, 
                            size=self.size_, 
                            iter=self.iter_,
                            sg=not self.is_cbow_, seed=self.random_state)

        return self
    
    def has(self, word):
        return word in self.w2v

    def transform(self, X):
        """
        X: a list of words
        """
        if self.w2v is None:
            raise Exception('model not fitted')
        return np.array([self.w2v[w] if w in self.w2v else np.zeros(self.size_) for w in X ])
    


C:\Users\Masha\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [24]:
%%time
# here we exploit that word2vec is an unsupervised learning algorithm
# so we can train it on the whole dataset (subject to discussion)

sentences_list = [x.strip() for x in ' '.join(df.word).split('.')]

w2v_cbow = Word2VecWrapper(window=5, negative=5, size=300, iter=300, is_cbow=True, random_state=SEED)
w2v_cbow.fit(sentences_list)

Wall time: 32.4 s


In [12]:
%%time
# baseline 1 
# random labels
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyClassifier


columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

model = Pipeline([
    ('enc', OneHotEncoder()),
    ('est', DummyClassifier(random_state=SEED)),
])

model.fit(df_train[columns], y_train)

print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))


train 0.05887736725599869
test 0.060439542712750365
CPU times: user 104 ms, sys: 13.3 ms, total: 118 ms
Wall time: 122 ms


In [16]:
%%time
# baseline 2 
# pos features + one hot encoding + logistic regression
from sklearn.preprocessing import OneHotEncoder


columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

model = Pipeline([
    ('enc', OneHotEncoder()),
    ('est', LogisticRegressionCV(Cs=5, cv=5, n_jobs=-1, scoring='f1_macro', 
                             penalty='l2', solver='newton-cg', multi_class='multinomial', random_state=SEED)),
])

model.fit(df_train[columns], y_train)

print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))

train 0.46639500282346874
test 0.39660981421559566
CPU times: user 3min 14s, sys: 13.1 s, total: 3min 27s
Wall time: 11min 30s


/Users/denaas/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/Users/denaas/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/denaas/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import LinearSVC
import scipy.sparse as sp

embeding = w2v_cbow
encoder_pos = OneHotEncoder()
X_train = sp.hstack([
    embeding.transform(df_train.word),
    embeding.transform(df_train['next-word']),
    embeding.transform(df_train['next-next-word']),
    embeding.transform(df_train['prev-word']),
    embeding.transform(df_train['prev-prev-word']),
    encoder_pos.fit_transform(df_train[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])
X_test = sp.hstack([
    embeding.transform(df_test.word),
    embeding.transform(df_test['next-word']),
    embeding.transform(df_test['next-next-word']),
    embeding.transform(df_test['prev-word']),
    embeding.transform(df_test['prev-prev-word']),
    encoder_pos.transform(df_test[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])


In [13]:
%%time
# baseline 3
# use word2vec cbow embedding + baseline 2 + svm

model = model_selection.GridSearchCV(LinearSVC(penalty='l2', multi_class='ovr', random_state=SEED), 
                                    {'C': np.logspace(-4, 0, 5)}, 
                                    cv=3, scoring='f1_macro', n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

print('train', metrics.f1_score(y_train, model.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(X_test), average='macro'))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  5.0min finished
/Users/denaas/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


train 0.95846030477073
test 0.8122113864662406
CPU times: user 1min 56s, sys: 5.51 s, total: 2min 1s
Wall time: 7min 2s


### RandomForest на обычных признаках

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

In [12]:
columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

model = Pipeline([
    ('enc', OneHotEncoder()),
    ('est', RandomForestClassifier(random_state=SEED))])

model.fit(df_train[columns], y_train)

print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))

train 0.7428021132450222
test 0.5773160505857489


Качество не очень.

### Подберем параметры для этой модели

In [13]:
%%time

columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

grid = {'n_estimators': [5, 20, 150, 400],
       'max_depth': [5, 40, 100, None],
       'min_samples_leaf': [1, 50, 200],
       'criterion': ['gini', 'entropy']}

model = Pipeline([
    ('enc', OneHotEncoder()),
    ('est', GridSearchCV(RandomForestClassifier(random_state=SEED), param_grid=grid, cv=5, n_jobs=-1, scoring='f1_macro', verbose=5))])

model.fit(df_train[columns], y_train)

print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 37.7min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 40.3min finished


train 0.7448336599801537
test 0.59833922231184
Wall time: 40min 27s


Чуть лучше, но все еще плохо.

### RandomForest на эмбедингах

In [32]:
model = RandomForestClassifier(random_state=SEED)

model.fit(X_train, y_train)

print('train', metrics.f1_score(y_train, model.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(X_test), average='macro'))

train 0.9885327458519465
test 0.8641661994055895


Отлично, побили бейзлайн.

### Подберем для этой модели признаки

In [33]:
%%time

columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

grid = {'n_estimators': [5, 20, 150, 400],
       'max_depth': [5, 40, 100, None],
       'min_samples_leaf': [1, 50, 200],
       'criterion': ['gini', 'entropy']}

model = GridSearchCV(RandomForestClassifier(random_state=SEED), param_grid=grid, cv=5, n_jobs=-1, scoring='f1_macro', verbose=5)

model.fit(X_train, y_train)

print('train', metrics.f1_score(y_train, model.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(X_test), average='macro'))

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 37.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 357.2min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 663.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 1776.3min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 1896.2min finished


train 0.9942245933151287
test 0.8664331619587545
Wall time: 1d 8h 42min 53s


Еще чуть-чуть повысили качество.

#### Итоговые параметры

In [34]:
model.best_params_

{'criterion': 'gini',
 'max_depth': 100,
 'min_samples_leaf': 1,
 'n_estimators': 150}